In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict

In [32]:
###################################
# A Class to store battery functionaly such as revenue, charge and discharge
# periods, charge and discharge spot prices, charge and discharge market dispatch
#
# NOTE : IF THERE IS ANY CHANGE IN CALCULATION, CHANGE CALCULATION MARKED BY (BATTERY CALCULATION).
#
# Consists of:
#      - charge_period : period when it should charge.
#      - discharge_period : period when it should discharge.
#      - charge_price : spot price given charging period.
#      - discharge_price : spot price given discharging period.
#      - charge_market_dispatch : set amount of market dispatch given charging period.
#      - discharge_market_dispatch : set amount of market dispatch given discharging period.
# 
# Functions:
#      - ComputeRevenue: To calculate revenue given discharge and charge period pairs.
#      - Setting : Set all the battery functionalities. (BATTERY CALCULATION)
#      - FirstOptimisation : Ensure that energy are not wasted or not used. (BATTERY CALCULATION)
#      - SecondOptimisation : Ensure that to always charge the highest price less and discharge
#                             highest price more. (BATTERY CALCULATION)
#
# Created by: Gilbert
###################################

OPENING = 0
CLOSING = 1

class Battery:
    # Battery Specifications
    mlf = 0.991                 # Marginal Loss Factor
    battery_capacity = 580      # Battery Capacity
    battery_power = 300         # Battery Power
    charge_efficiency = 0.9     # Charge Efficiency
    discharge_efficiency = 0.9  # Discharge Efficiency
        
    def __init__(self, charge_period, charge_spot_price,
                 discharge_period, discharge_spot_price): 
        # Charge and Discharge Period
        self.charge_period = charge_period
        self.discharge_period = discharge_period
        
        # Spot Price during Charge and Discharge
        self.charge_price = charge_spot_price
        self.discharge_price = discharge_spot_price
    
    ########################################################################################    
                
    def Revenue(self):
        if self.charge_market_dispatch == [] or self.discharge_market_dispatch == []:
            return 0
        
        # Spot Prices
        charge_sp = np.array(self.charge_price)[:, 1]
        discharge_sp = np.array(self.discharge_price)[:, 1]
        
        # Market Dispatches
        charge_md = np.array(self.charge_market_dispatch).T
        discharge_md = np.array(self.discharge_market_dispatch).T
        
        # Revenues
        charge_revenue = (charge_sp @ charge_md) * (1 / self.mlf)
        discharge_revenue = (discharge_sp @ discharge_md) * (self.mlf)
        
        return discharge_revenue + charge_revenue
    
    ########################################################################################
    
    def Setting(self):
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        self.charge_raw_power = ['' for i in range(len_charge)]
        self.discharge_raw_power = ['' for i in range(len_discharge)]
        
        self.charge_market_dispatch = ['' for i in range(len_charge)]
        self.discharge_market_dispatch = ['' for i in range(len_discharge)]
        
        self.charge_capacity = [[0, 0] for i in range(len_charge)]
        self.discharge_capacity = [[0, 0] for i in range(len_discharge)]
        
        # CHARGE PERIOD --------------------------------------------------------------------
        for t in range(len_charge):
            # RAW_POWER[t] = -MIN(BATTERY_POWER, (BATTERY_CAPACITY - OPENING_CAPACITY[t]) / CHARGE_EFFICIENCY * 2)
            self.charge_raw_power[t] = -min(battery_power, 
                                           (battery_cap - self.charge_capacity[t][OPENING]) / 
                                            self.charge_efficiency * 2)

            # MARKET_DISPATCH[t] = RAW_POWER / 2
            self.charge_market_dispatch[t] = self.charge_raw_power[t] / 2

            # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
            #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
            self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                        self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                        battery_cap))

            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_charge:
                self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------

        # Set DISCHARGE CAPACITY AT t = 0 to be the the last t of CHARGING CAPACITY
        self.discharge_capacity[0][OPENING] = self.charge_capacity[-1][CLOSING]

        for t in range(len_discharge):
            # RAW_POWER[t] = MIN(BATTERY_POWER, OPENING_CAPACITY[t] * 2)
            self.discharge_raw_power[t] = min(battery_power, self.discharge_capacity[t][OPENING] * 2)

            # MARKET_DISPATCH[t] = RAW_POWER[t] / 2 * DISCHARGE EFFICIENCY
            self.discharge_market_dispatch[t] = self.discharge_raw_power[t] / 2 * self.discharge_efficiency

            # CLOSING CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] -
            #                        MARKET_DISPATCH[t] * (1/DISCHARGE_EFFICIENCY), BATTERY_CAPACITY))
            self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] -
                                                            self.discharge_market_dispatch[t] * (1 / self.discharge_efficiency),
                                                            battery_cap))
            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_discharge:
                self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
    
    ########################################################################################
        
    def FirstOptimisation(self):
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        if (len_charge >= MAX_CHARGE_PERIOD and len_discharge >= MAX_DISCHARGE_PERIOD) or (len_charge == len_discharge):
            #md = 117
            #self.discharge_raw_power[-1] = min(md * 2 / self.discharge_efficiency, self.discharge_raw_power[-1])
            #self.discharge_market_dispatch[-1] = self.discharge_raw_power[-1] / 2 * self.discharge_efficiency
            return
        elif (len_charge - len_discharge == 1):
            self.discharge_capacity[-1][CLOSING] = 0
            # OPENING_CAPACITY[-1] = MARKET_DISPATCH / DISCHARGE_EFFICIENCY
            self.discharge_capacity[-1][OPENING] = self.discharge_market_dispatch[-1] / self.discharge_efficiency
            for t in range(1, len_discharge):
                # CLOSING_CAPACITY[-t - 1] = OPENING_CAPACITY[-t]
                self.discharge_capacity[-t - 1][CLOSING] = self.discharge_capacity[-t][OPENING]
                # OPENING_CAPACITY[-t - 1] = CLOSING_CAPACITY[-t - 1] + MARKET_DISPATCH[-t - 1] / DISCHARGE_EFFICIENCY
                self.discharge_capacity[-t - 1][OPENING] = self.discharge_capacity[-t - 1][CLOSING] + self.discharge_market_dispatch[-t - 1] / self.discharge_efficiency
            
            # CLOSING_CAPACITY[-1] = OPENING_CAPACITY[0]
            self.charge_capacity[-1][CLOSING] = self.discharge_capacity[0][OPENING]
            # MARKET_DISPATCH[-1] = -(CLOSING_CAPACITY[-1] - OPENING_CAPACITY[-1]) / CHARGE_EFFICIENCY
            self.charge_market_dispatch[-1] = -(self.charge_capacity[-1][CLOSING] - self.charge_capacity[-1][OPENING]) / self.charge_efficiency
            # RAW_POWER[-1] = MARKET_DISPATCH[-1] * 2
            self.charge_raw_power[-1] = self.charge_market_dispatch[-1] * 2
            
    ########################################################################################

    def SecondOptimisation(self):
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        # CHARGE PERIOD --------------------------------------------------------------------
        highest_price_index = np.array(self.charge_period).argmax(axis=0)[0]
        lowest_charge_index = self.charge_market_dispatch.index(max(self.charge_market_dispatch))    
        
        # IF THE HIGHEST CHARGING PRICE DOESN'T HAVE THE LOWEST CHARGING RATE, SWAP!
        if (highest_price_index != lowest_charge_index):
            tmp = self.charge_market_dispatch[highest_price_index]
            self.charge_market_dispatch[highest_price_index] = self.charge_market_dispatch[lowest_charge_index]
            self.charge_market_dispatch[lowest_charge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_charge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2
                self.charge_raw_power[t] = self.charge_market_dispatch[t] * 2

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                            self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_charge:
                    self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------
        lowest_price_index = np.array(self.discharge_period).argmax(axis=0)[0] 
        min_discharge_index = self.discharge_market_dispatch.index(min(self.discharge_market_dispatch))   
        
        # IF THE LOWEST DISCHARGING PRICE DOESN'T HAVE THE LOWEST DISCHARGING RATE, SWAP!
        if (lowest_price_index != min_discharge_index):
            tmp = self.discharge_market_dispatch[lowest_price_index]
            self.discharge_market_dispatch[lowest_price_index] = self.discharge_market_dispatch[min_discharge_index]
            self.discharge_market_dispatch[min_discharge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_discharge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2 / DISCHARGE_EFFICIENCY
                self.discharge_raw_power[t] = self.discharge_market_dispatch[t] * 2 / self.discharge_efficiency

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] - 
                                                            self.discharge_market_dispatch[t] / self.discharge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_discharge:
                    self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
                    
    #
    # REVERSE BATTERY
    #
    def ReverseSetting(self, discharge_capacity, charge_capacity):
        if len(self.charge_period) == len(self.discharge_period):
            self.EqualSetting(discharge_capacity, charge_capacity)
        else:
            self.NonEqualSetting(discharge_capacity, charge_capacity)
            
    def EqualSetting(self, discharge_capacity, charge_capacity):
        battery_power = self.battery_power
        battery_cap = np.amax(charge_capacity)
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        MAX_RAW_POWER = 300
        MAX_CHARGE_MD = MAX_RAW_POWER / 2
        MAX_DISCHARGE_MD = MAX_RAW_POWER / 2 * self.discharge_efficiency
        
        self.charge_capacity = charge_capacity
        self.discharge_capacity = discharge_capacity
        
        self.charge_raw_power = ['' for i in range(len_charge)]
        self.discharge_raw_power = ['' for i in range(len_discharge)]
        
        self.charge_market_dispatch = ['' for i in range(len_charge)]
        self.discharge_market_dispatch = ['' for i in range(len_discharge)]
        
        self.charge_capacity[-1][OPENING] = max(0, min(self.charge_capacity[-1][CLOSING] - MAX_CHARGE_MD * self.charge_efficiency, battery_cap))
                                                
        for t in range(1, len_charge):
            self.charge_capacity[-t -1][CLOSING] = self.charge_capacity[-t][OPENING]
            self.charge_capacity[-t -1][OPENING] = max(0, min(self.charge_capacity[-t -1][CLOSING] - MAX_CHARGE_MD * self.charge_efficiency, battery_cap))
            
            self.charge_market_dispatch[-t] = -min((self.charge_capacity[-t][CLOSING] - 
                                                    self.charge_capacity[-t][OPENING]) / self.charge_efficiency, MAX_CHARGE_MD)
            self.charge_raw_power[-t] = max(-MAX_RAW_POWER, self.charge_market_dispatch[-t] * 2)
            
        self.charge_market_dispatch[0] = -min((self.charge_capacity[0][CLOSING] - 
                                               self.charge_capacity[0][OPENING]) / self.charge_efficiency, MAX_CHARGE_MD)
        self.charge_raw_power[0] = max(-MAX_RAW_POWER, self.charge_market_dispatch[0] * 2)  
        
        self.discharge_capacity[-1][CLOSING] = self.charge_capacity[0][OPENING]
        self.discharge_capacity[-1][OPENING] = min(battery_cap, self.discharge_capacity[-1][CLOSING] + MAX_DISCHARGE_MD / self.discharge_efficiency)
        self.discharge_market_dispatch[-1] = min(MAX_DISCHARGE_MD, (self.discharge_capacity[-1][OPENING] - self.discharge_capacity[-1][CLOSING]) * self.discharge_efficiency) 
        self.discharge_raw_power[-1] = min(MAX_RAW_POWER, self.discharge_market_dispatch[-1] * 2 / self.discharge_efficiency)
        for t in range(1, len_discharge):
            self.discharge_capacity[-t -1][CLOSING] = self.discharge_capacity[-t][OPENING]
            self.discharge_capacity[-t -1][OPENING] = min(battery_cap, self.discharge_capacity[-t -1][CLOSING] + MAX_DISCHARGE_MD / self.discharge_efficiency)
            self.discharge_market_dispatch[-t-1] = min(MAX_DISCHARGE_MD, (self.discharge_capacity[-t -1][OPENING] - self.discharge_capacity[-t -1][CLOSING]) * self.discharge_efficiency) 
            self.discharge_raw_power[-t -1] = min(MAX_RAW_POWER, self.discharge_market_dispatch[-t -1] * 2 / self.discharge_efficiency)    
        
    def NonEqualSetting(self, discharge_capacity, charge_capacity):
        battery_power = self.battery_power
        battery_cap = np.amax(charge_capacity)
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        MAX_RAW_POWER = 300
        MAX_CHARGE_MD = MAX_RAW_POWER / 2
        MAX_DISCHARGE_MD = MAX_RAW_POWER / 2 * self.discharge_efficiency
        
        self.charge_capacity = charge_capacity
        self.discharge_capacity = discharge_capacity
        
        self.charge_raw_power = ['' for i in range(len_charge)]
        self.discharge_raw_power = ['' for i in range(len_discharge)]
        
        self.charge_market_dispatch = ['' for i in range(len_charge)]
        self.discharge_market_dispatch = ['' for i in range(len_discharge)]
                
        self.discharge_capacity[0][CLOSING] = max(0, min(self.discharge_capacity[0][OPENING] - 
                                                         MAX_DISCHARGE_MD / self.discharge_efficiency, battery_cap))
        #self.discharge_market_dispatch[0] = min((self.discharge_capacity[0][OPENING] - 
        #                                        self.discharge_capacity[0][CLOSING]) * self.discharge_efficiency, MAX_DISCHARGE_MD)
        #self.discharge_raw_power[0] = min(MAX_RAW_POWER, self.discharge_market_dispatch[0] * 2 / 0.9)
        
        for t in range(1, len_discharge):
            self.discharge_capacity[t][OPENING] = self.discharge_capacity[t - 1][CLOSING]
            self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] - MAX_DISCHARGE_MD / self.discharge_efficiency, battery_cap))
            self.discharge_market_dispatch[t-1] = min((self.discharge_capacity[t-1][OPENING] - 
                                                    self.discharge_capacity[t-1][CLOSING]) * self.discharge_efficiency, MAX_DISCHARGE_MD)
            self.discharge_raw_power[t-1] = min(MAX_RAW_POWER, self.discharge_market_dispatch[t-1] * 2 / 0.9)
        
        self.discharge_market_dispatch[-1] = min((self.discharge_capacity[-1][OPENING] - 
                                                self.discharge_capacity[-1][CLOSING]) * self.discharge_efficiency, MAX_DISCHARGE_MD)
        self.discharge_raw_power[-1] = min(MAX_RAW_POWER, self.discharge_market_dispatch[-1] * 2 / 0.9)
        
        self.charge_capacity[0][OPENING] = self.discharge_capacity[-1][CLOSING]
        self.charge_market_dispatch[0] = min(self.charge_capacity[0][CLOSING] - 
                                             self.charge_capacity[0][OPENING] * self.charge_efficiency, MAX_CHARGE_MD)
        
        self.charge_raw_power[0] = min(MAX_RAW_POWER, self.charge_market_dispatch[0] * 2)
        for t in range(1, len_charge):
            self.charge_capacity[t - 1][CLOSING] = min(battery_cap, self.charge_capacity[t-1][OPENING] + MAX_CHARGE_MD * self.charge_efficiency)
            self.charge_capacity[t][OPENING] = self.charge_capacity[t - 1][CLOSING]
            
            self.charge_market_dispatch[t-1] = -min((self.charge_capacity[t-1][CLOSING] - 
                                                 self.charge_capacity[t-1][OPENING]) / self.charge_efficiency, MAX_CHARGE_MD)
            self.charge_raw_power[t-1] = max(-MAX_RAW_POWER, self.charge_market_dispatch[t-1] * 2)
            
        self.charge_market_dispatch[-1] = -min((self.charge_capacity[-1][CLOSING] - 
                                                 self.charge_capacity[-1][OPENING]) / self.charge_efficiency, MAX_CHARGE_MD)
        self.charge_raw_power[-1] = max(-MAX_RAW_POWER, self.charge_market_dispatch[-1] * 2)

### Normal Battery Helper

In [33]:
EMPTY = ' '
###################################
# A function to get spot prices based on selected regions.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - selected_periods : selected period for charging or discharging.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#
# Return:
#      - List of spot prices given period
#
# Created by: Gilbert
###################################
def GetSpotPrice(data, selected_periods = False, region = 'VIC'):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    spot_price = np.array(spot_price)
    
    # Find the spot prices from selected region. Periods are
    # index + 1, therefore to use the index we need to subtract
    # it by 1.
    if (selected_periods):
        retrieved_prices = []
        for period in selected_periods:
            # append(minimum or maximum ranking, spot_price[index])
            retrieved_prices.append((period[1], spot_price[period[1] - 1]))
        return retrieved_prices
    else:
        return spot_price

########################################################################################

###################################
# A function to find minimum and maximum point rank given threshold.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#      - buy_threshold : maximum number of buying point, default has been set to optimise Checkpoint 3.
#      - sell_threshold : maximum number of selling point, default has been set to optimise Checkpoint 3.
#
# Return:
#      - List of selected minimum point, list of selected maximum point
#
# Efficiency: O(3N + NLogN) = O(NLogN)
#
# Created by: Gilbert
###################################
def GetMinMax(data, region = 'VIC', buy_threshold = 5, sell_threshold = 4):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    price = np.array(spot_price)
    minimum_price = np.argsort(price, kind = 'merge*sort') # (O(NlogN)), mergesort the minimum prices.
    maximum_price = minimum_price[::-1][:len(price)] # (O(N)), maximum is the reverse order of minimum.
    
    selected_min_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    selected_max_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    
    # Select the lowest price spot over the given
    # buy_threshold as the minimum buying point.
    i = 0
    for b_t in range(buy_threshold):
        selected_min_price[minimum_price[i]] = b_t + 1
        i += 1
        
    # Select the highest price spot over the given
    # sell_threshold as the maximum selling point.
    i = 0
    for s_t in range(sell_threshold):
        selected_max_price[maximum_price[i]] = s_t + 1
        i += 1
        
    return selected_min_price, selected_max_price

########################################################################################

###################################
# A function to Find Battery Charge and Discharge pairs in backward order.
# Backward order from 48th period to the 1st.
#
# Parameters:
#      - buy_period : Selected minimum price point as it will be where we buy energy for charging.
#      - sell_period : Selected maximum price point as it will be where we sell energy for discharging.
#
# Return:
#      - List of battery class pairs
#
# Efficiency: O(N)
#
# Created by: Gilbert
###################################
def FindBatteryPairs(buy_period, sell_period):
    MAX_SELL_PERIOD = 4 # MAXIMUM SELLING PERIOD PER PAIR
    MAX_BUY_PERIOD = 5 # MAXIMUM BUYING PERIOD PER PAIR
    
    period = len(buy_period)
    
    battery = []
    sell = OrderedDict() # Initialise battery selling point. (Ordered Dictionary)
    buy = OrderedDict() # Initialise battery buying point. (Orderered Dictionary)
    
    # Iterate over the whole period backwards
    for p in range(period - 1, -1, -1):
        # If maximum selling point is not empty, add (order, period)
        # as key-value pair into the OrderedDict.
        if sell_period[p] != EMPTY and buy_period[p] == EMPTY:
            # If battery buying point period is less MAXIMUM SELLING 
            # PERIOD PER PAIR, add new period.
            if len(sell) < MAX_SELL_PERIOD:
                sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1
            # else, if battery selling point is full and there is 
            # higher maximum selling point then remove the lowest
            # selling point and add the new one into Dictionary.    
            else:
                max_key = max(sell, key=int)
                if sell_period[p] < max_key:
                    sell.pop(max_key)
                    sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1

        # If battery selling point is not empty and minimum buying 
        # point is not empty.
        if len(sell) != 0 and sell_period[p] == EMPTY and buy_period[p] != EMPTY :
            # If battery buying point period is less MAXIMUM BUYING 
            # PERIOD PER PAIR, add new period.
            if len(buy) < MAX_BUY_PERIOD and len(buy) < math.ceil(len(sell) * 1.25):
                buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
            # else, if battery buying point is full and there is 
            # lower minimum buying point then remove the highest
            # buying point and add the new one into Dictionary.
            else:
                max_key = max(buy, key=int)
                if buy_period[p] < max_key:
                    buy.pop(max_key)
                    buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
        # If the next period is not empty and battery buying point
        # is not empty then battery charge-discharge pair has been
        # created.
        # Reinitialise a new battery setup.
        if sell_period[p - 1] != EMPTY and len(buy) != 0:
            battery.append([list(sell.items()), list(buy.items())])
            sell = OrderedDict()
            buy = OrderedDict()
    # Add the last battery charge-discharge pair occuring 
    # before 1st period.
    battery.append([list(sell.items()), list(buy.items())]) 

    # Check whether there is too many selling points, then
    # remove selling point until the number of selling points
    # is equal to the number of buying points while removing
    # the lowest selling point.
    for b in battery:
        sell_tmp = np.array(b[0])
        buy_tmp = b[1]
        while len(sell_tmp) > len(buy_tmp): 
            row = 0
            index = np.where(sell_tmp[:,0] == sell_tmp[:,0].max())[0][0]
            sell_tmp = np.delete(sell_tmp, index, axis = row)
        b[0] = sell_tmp.tolist() # Change numpy array to list

    return battery

########################################################################################

###################################
# A function to set optimal charge and discharge amount of battery pairs.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - battery_pairs : list of all battery class pairs.
#
# Return:
#      - List of all battery class pairs
#
# Efficiency: O(N)
#
# Created by: Gilbert
###################################
def SetChargeDischarge(data, battery_pairs, region = "VIC"):    
    all_batteries = []
    
    battery_pairs = battery_pairs[::-1]
    for b in battery_pairs:
        sell_period = b[0][::-1] # Reverse the order
        buy_period = b[1][::-1] # Reverse the order
        if len(sell_period)!= 0 or len(buy_period) != 0:
            sell_price = GetSpotPrice(data, sell_period, region = region)
            buy_price = GetSpotPrice(data, buy_period, region = region)

            battery = Battery(buy_period, buy_price, sell_period, sell_price)
            battery.Setting()
            battery.FirstOptimisation()
            battery.SecondOptimisation()

            all_batteries.append(battery)

    return all_batteries

########################################################################################

###################################
# A function to calculate daily revenue.
#
# Parameters:
#      - all_batteries : List of battery class pairs.
#
# Return:
#      - Daily revenues
#
# Created by: Gilbert
###################################
def ComputeDailyRevenue(all_batteries):
    revenues = 0
    for battery in all_batteries:
        revenues += battery.Revenue()
    return revenues

########################################################################################

### Normal Model

In [34]:
###################################
# A function to optimise battery charging and discharging period. This is where 
# mainly the optimisations are performed with the helper functions.
#
# Parameters:
#      - daily_data : the targeted daily dataset, minimum dataset length of 48.
#
# Return:
#      - raw_power : List of Daily Raw Power for charging and discharging
#      - market_dispatch : List of Daily Market Dispatch for charging and discharging
#      - opening_capacity : List of Daily Opening Capacity for charging and discharging
#      - closing_capacity : List of Daily Closing Capacity for charging and discharging
#
# Efficiency: O(N^2 * NLogN) = O(N^3LogN)
# Created by: Gilbert
###################################
def PeriodOptimisation(given_data, region = "VIC"):
    best_batteries = OrderedDict() # Initialise an Ordered Dictionary
    
    # TODO: OPTIMISE EFFICIENCY HERE! Make this at least < N^2
    # Iterate over all possible combinations of battery pairs. (O(N^2))
    period = len(given_data)
    count = 0
    for s in range(1, period + 1):
        for b in range(1 , period + 1 - s): # change this to range(1, period - s + 1) to reduce by half. 
            # Get the minimum and maximum price based on the given threshold
            min_price, max_price = GetMinMax(given_data, buy_threshold = b, sell_threshold = s, region = region)
            # Get the battery pairs based on minimum and maximum price
            battery_pairs = FindBatteryPairs(min_price, max_price)
            # Get battery optimisation for the selected threshold
            all_batteries = SetChargeDischarge(given_data, battery_pairs, region = region)
            # Compute daily revenues of selected battery combinations
            dailyrev = ComputeDailyRevenue(all_batteries)
            #print((b, s), dailyrev)
            if dailyrev < 0:
                break
            count += 1
            # Insert revenue as key, batteries combination and threshold as value
            if dailyrev not in best_batteries:
                best_batteries[dailyrev] = (all_batteries, (b, s))
                
    # Find the highest revenue amongst possible combinations in that day
    best_revenue = max(best_batteries)
    #best_threshold = best_batteries[best_revenue][1]
    #print(best_threshold)
    battery = best_batteries[best_revenue][0] # The Best battery combinations
    
    # Initialise raw_power, market_dispatch, opening_capacity and closing capacity
    raw_power = [0 for i in range(period)]
    market_dispatch = [0 for i in range(period)]
    opening_capacity = [0 for i in range(period)]
    closing_capacity = [0 for i in range(period)]
    
    # Iterate over battery combinations to set raw_power, market_dispatch,
    # opening_capacity, closing_capacity into an array to be prepared for 
    # merging with the dataset.
    for b in battery:   
        # Charging Period
        for cp in range(len(b.charge_period)):
            raw_power[b.charge_period[cp][1] - 1] = b.charge_raw_power[cp]
            market_dispatch[b.charge_period[cp][1] - 1] = b.charge_market_dispatch[cp]
            opening_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][0]
            closing_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][1]
        # Discharge Period
        for dp in range(len(b.discharge_period)):
            raw_power[b.discharge_period[dp][1] - 1] = b.discharge_raw_power[dp]
            market_dispatch[b.discharge_period[dp][1] - 1] = b.discharge_market_dispatch[dp]
            opening_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][0]
            closing_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][1]
    
    # Formatting the opening and closing capacity.
    for i in range(1, len(opening_capacity)):
        if closing_capacity[i - 1] != 0.0 and opening_capacity[i] == 0.0:
            opening_capacity[i] = closing_capacity[i - 1]
            closing_capacity[i] = opening_capacity[i]
                
    return raw_power, market_dispatch, opening_capacity, closing_capacity, best_revenue

# A Main function to run the algorithm.
# FixedOptimisation : This algorithm generates over a period given that there is no dependency.
def FixedOptimisation(data, period = 48, region = 'VIC'):
    raw_power = []
    market_dispatch = []
    opening_capacity = []
    closing_capacity = []

    start = 0
    end = period 
    
    # Given the data iterate over the given period.
    while end <= len(data):
        tmp_data = data.iloc[start:end, :]
        daily_rp, daily_md, daily_oc, daily_cc, _ = PeriodOptimisation(tmp_data) # Optimise the period given
        raw_power.extend(daily_rp)
        market_dispatch.extend(daily_md)
        opening_capacity.extend(daily_oc)
        closing_capacity.extend(daily_cc)

        start += period
        end += period

    data['Raw Power (MW)'] = pd.Series(raw_power)
    data['Market Dispatch (MWh)'] = pd.Series(market_dispatch)
    data['Opening Capacity (MWh)'] = pd.Series(opening_capacity)
    data['Closing Capacity (MWh)'] = pd.Series(closing_capacity)
    
    return data

### Reverse Battery Helper

In [35]:
def setStatus(dataframe):
    status = []
    for i in range(len(dataframe)):
        market_d = dataframe.loc[i, 'Market Dispatch (MWh)']
        opening_c = dataframe.loc[i, 'Opening Capacity (MWh)']
        closing_c = dataframe.loc[i, 'Closing Capacity (MWh)']
        
        if market_d > 0:
            status.append('Discharge')
        elif market_d < 0:
            status.append('Charge')
        elif market_d == 0 and (opening_c != 0 or closing_c != 0):
            status.append('Between')
        else:
            status.append('Nothing')
    return status

def selectBetweenPeriod(dataframe):
    betweenIndex = dataframe[dataframe['Status'] == 'Between'].index
    period = []
    tmp = [betweenIndex[0]]
    for i in range(1, len(betweenIndex)):
        prev = betweenIndex[i - 1]
        curr = betweenIndex[i]
        if curr - prev > 1:
            period.append(tmp)
            tmp = [curr]
        else:
            tmp.append(curr)
            
    period.append(tmp)
    
    tmp = []
    for p in range(len(period)):
        if len(period[p]) >= 2:
            tmp.append([period[p][0], period[p][-1]])
    period = tmp
    
    timePeriod = []
    for p in period:
        timePeriod.append((dataframe.iloc[p[0], 0], dataframe.iloc[p[1], 0]))
    
    return timePeriod

def GetCapacity(data, selected_periods = False):
    opening_cap = np.array(data['Opening Capacity (MWh)'])
    closing_cap = np.array(data['Closing Capacity (MWh)'])
    
    if (selected_periods):
        retrieved_cap = []
        for period in selected_periods:
            retrieved_cap.append([opening_cap[period[1] - 1], closing_cap[period[1] - 1]])
        return retrieved_cap
    else:
        return [opening_cap, closing_cap]
    
def findBatteryPairsReverse(buy_period, sell_period, dataframe):
    MAX_SELL_PERIOD = 4 # MAXIMUM SELLING PERIOD PER PAIR
    MAX_BUY_PERIOD = 5 # MAXIMUM BUYING PERIOD PER PAIR
    
    period = len(buy_period)
    
    cap = GetCapacity(dataframe)
    max_cap = max(cap[0])
    #print(max_cap,max_cap//150 + 1)
    battery = []
    sell = OrderedDict() # Initialise battery selling point. (Ordered Dictionary)
    buy = OrderedDict() # Initialise battery buying point. (Orderered Dictionary)
    
    for p in range(period - 1):
        if sell_period[p] != EMPTY and buy_period[p] == EMPTY:
            if len(sell) < MAX_SELL_PERIOD and len(sell) < max_cap // 150 + 1:
                #print("YES")
                sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1
            else:
                max_key = max(sell, key=int)
                if sell_period[p] < max_key:
                    sell.pop(max_key)
                    sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1

        if len(sell) != 0 and sell_period[p] == EMPTY and buy_period[p] != EMPTY:
            if len(buy) < MAX_BUY_PERIOD and len(buy) < math.ceil(len(sell) * 1.25) and len(buy) < max_cap // 135 + 1:
                buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
            else:
                max_key = max(buy, key=int)
                if buy_period[p] < max_key:
                    buy.pop(max_key)
                    buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
        if sell_period[p + 1] != EMPTY and len(buy) != 0:
                battery.append([list(sell.items()), list(buy.items())])
                sell = OrderedDict()
                buy = OrderedDict()
    battery.append([list(sell.items()), list(buy.items())]) 

    for b in battery:
        sell_tmp = np.array(b[0])
        buy_tmp = b[1]
        while len(sell_tmp) > len(buy_tmp): 
            row = 0
            index = np.where(sell_tmp[:,0] == sell_tmp[:,0].max())[0][0]
            sell_tmp = np.delete(sell_tmp, index, axis = row)
        b[0] = sell_tmp.tolist() # Change numpy array to list
        if len(b[0]) == 0 or len(b[1]) == 0:
            battery.remove(b)

    return battery

def SetChargeDischargeReverse(data, battery_pairs, region = "VIC"):
    opening_cap = data['Opening Capacity (MWh)']
    closing_cap = data['Closing Capacity (MWh)']
    
    all_batteries = []
    
    for b in battery_pairs:
        sell_period = b[0] 
        buy_period = b[1] 
        if len(sell_period)!= 0 or len(buy_period) != 0:
            sell_price = GetSpotPrice(data, sell_period, region = region)
            buy_price = GetSpotPrice(data, buy_period, region = region)
            battery = Battery(buy_period, buy_price, sell_period, sell_price)

            sell_cap = GetCapacity(data, sell_period)
            buy_cap = GetCapacity(data, buy_period)
            battery.ReverseSetting(sell_cap, buy_cap)
            battery.SecondOptimisation()

            all_batteries.append(battery)

    return all_batteries

### Reverse Model

In [36]:
def localOptimisation(dataframe):
    period = len(dataframe)
    best_batteries = OrderedDict()
    
    for s in range(1, len(dataframe) + 1):
        for b in range(1, len(dataframe) + 1 - s):
            min_price, max_price = GetMinMax(dataframe, buy_threshold = b, sell_threshold = s)
            battery_pairs = findBatteryPairsReverse(min_price, max_price, dataframe)
            all_batteries = SetChargeDischargeReverse(dataframe, battery_pairs)
            dailyrev = ComputeDailyRevenue(all_batteries)
            if dailyrev not in best_batteries:
                best_batteries[dailyrev] = (all_batteries, (b, s))

    # Find the highest revenue amongst possible combinations in that day
    best_revenue = max(best_batteries)
    best_threshold = best_batteries[best_revenue][1]
    battery = best_batteries[best_revenue][0] # The Best battery combinations

    # Initialise raw_power, market_dispatch, opening_capacity and closing capacity
    raw_power = [0 for i in range(period)]
    market_dispatch = [0 for i in range(period)]
    opening_capacity = dataframe.loc[:, 'Opening Capacity (MWh)'].to_numpy()
    closing_capacity = dataframe.loc[:, 'Closing Capacity (MWh)'].to_numpy()
    
    # Iterate over battery combinations to set raw_power, market_dispatch,
    # opening_capacity, closing_capacity into an array to be prepared for 
    # merging with the dataset.
    for b in battery:   
        # Charging Period
        for cp in range(len(b.charge_period)):
            raw_power[b.charge_period[cp][1] - 1] = b.charge_raw_power[cp]
            market_dispatch[b.charge_period[cp][1] - 1] = b.charge_market_dispatch[cp]
            opening_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][0]
            closing_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][1]
        # Discharge Period
        for dp in range(len(b.discharge_period)):
            raw_power[b.discharge_period[dp][1] - 1] = b.discharge_raw_power[dp]
            market_dispatch[b.discharge_period[dp][1] - 1] = b.discharge_market_dispatch[dp]
            opening_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][0]
            closing_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][1]
            
       
    # Formatting the opening and closing capacity.
    change = 0
    for i in range(1, len(opening_capacity)):
        if closing_capacity[i - 1] != opening_capacity[i]:
            opening_capacity[i] = closing_capacity[i - 1]
            closing_capacity[i] = opening_capacity[i]

    return raw_power, market_dispatch, opening_capacity, closing_capacity, best_revenue

def localMaximisation(dataframe, timePeriod):
    tmp_df = dataframe.copy()
    for period in timePeriod:
        start_t = period[0]
        end_t = period[1]

        data_interval = tmp_df.loc[(tmp_df['Time (UTC+10)'] >= start_t) & 
                                      (tmp_df['Time (UTC+10)'] <= end_t)]
        
        start_index = data_interval.index[0]

        price = GetSpotPrice(data_interval).tolist()
        min_price = (price.index(min(price)), min(price))
        max_price = (price.index(max(price)), max(price))
        
        raw_power, market_dispatch, opening_capacity, closing_capacity, _ = localOptimisation(data_interval)
        for i in range(len(raw_power)):
            tmp_df.loc[i + start_index, 'Raw Power (MW)'] = raw_power[i]
            tmp_df.loc[i + start_index, 'Market Dispatch (MWh)'] = market_dispatch[i]
            tmp_df.loc[i + start_index, 'Opening Capacity (MWh)'] = opening_capacity[i]
            tmp_df.loc[i + start_index, 'Closing Capacity (MWh)'] = closing_capacity[i]

    return tmp_df

In [72]:
def fullMaximisation(dataframe, period):
    tmp_df = dataframe.copy()
    raw_power = []
    market_dispatch = []
    opening_capacity = []
    closing_capacity = []

    start = 0
    end = period 
    
    # Given the data iterate over the given period.
    while end <= len(tmp_df):
        tmp_data = tmp_df.iloc[start:end, :]
        daily_rp, daily_md, daily_oc, daily_cc, _ = PeriodOptimisation(tmp_data) # Optimise the period given
        raw_power.extend(daily_rp)
        market_dispatch.extend(daily_md)
        opening_capacity.extend(daily_oc)
        closing_capacity.extend(daily_cc)

        start += period
        end += period

    tmp_df['Raw Power (MW)'] = pd.Series(raw_power)
    tmp_df['Market Dispatch (MWh)'] = pd.Series(market_dispatch)
    tmp_df['Opening Capacity (MWh)'] = pd.Series(opening_capacity)
    tmp_df['Closing Capacity (MWh)'] = pd.Series(closing_capacity)

    status = setStatus(tmp_df)
    tmp_df['Status'] = pd.Series(status)
    
    betweenPeriod = selectBetweenPeriod(tmp_df)
    newData = localMaximisation(tmp_df, betweenPeriod)
    
    return newData

### Original Data

In [13]:
market_data = pd.read_excel("../../raw_data/market_data.xlsx")
market_data.drop(index=market_data.index[0], axis=0, inplace=True)
market_data = market_data.reset_index(drop = True)

In [19]:
vic_spot_price = market_data[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]].copy()
vic_spot_price

,Time (UTC+10),Regions VIC Trading Price ($/MWh)
0,2018-01-01 00:30:00,92.46
1,2018-01-01 01:00:00,87.62
2,2018-01-01 01:30:00,73.08
3,2018-01-01 02:00:00,70.18
4,2018-01-01 02:30:00,67.43
...,...,...
63451,2021-08-14 22:00:00,49.93
63452,2021-08-14 22:30:00,62.86
63453,2021-08-14 23:00:00,32.26
63454,2021-08-14 23:30:00,25.10


In [62]:
data_interval = vic_spot_price.loc[(vic_spot_price['Time (UTC+10)'] >= '2021-07-01 00:30:00') & \
                                   (vic_spot_price['Time (UTC+10)'] <= '2021-08-15 00:00:00')].copy()
data_interval = data_interval.reset_index(drop = True)
data_interval

,Time (UTC+10),Regions VIC Trading Price ($/MWh)
0,2021-07-01 00:30:00,90.51
1,2021-07-01 01:00:00,73.91
2,2021-07-01 01:30:00,33.79
3,2021-07-01 02:00:00,43.57
4,2021-07-01 02:30:00,45.90
...,...,...
2155,2021-08-14 22:00:00,49.93
2156,2021-08-14 22:30:00,62.86
2157,2021-08-14 23:00:00,32.26
2158,2021-08-14 23:30:00,25.10


In [38]:
fullmodel = fullMaximisation(data_interval, period = 48)

In [39]:
fullmodel

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Status
0,2021-07-01 00:30:00,90.51,0.000000,0.000000,0.0,0.0,Nothing
1,2021-07-01 01:00:00,73.91,0.000000,0.000000,0.0,0.0,Nothing
2,2021-07-01 01:30:00,33.79,-300.000000,-150.000000,0.0,135.0,Charge
3,2021-07-01 02:00:00,43.57,-88.888889,-44.444444,135.0,175.0,Charge
4,2021-07-01 02:30:00,45.90,0.000000,0.000000,175.0,175.0,Between
...,...,...,...,...,...,...,...
2155,2021-08-14 22:00:00,49.93,-300.000000,-150.000000,0.0,135.0,Charge
2156,2021-08-14 22:30:00,62.86,270.000000,121.500000,135.0,0.0,Discharge
2157,2021-08-14 23:00:00,32.26,0.000000,0.000000,0.0,0.0,Nothing
2158,2021-08-14 23:30:00,25.10,0.000000,0.000000,0.0,0.0,Nothing


In [45]:
max_charge = []
min_charge = []
flags = []
for i in range(len(fullmodel)):
    max_c = -min(300, (580 - fullmodel.loc[i, 'Opening Capacity (MWh)'])/0.9 * 2)
    min_c = min(300, fullmodel.loc[i, 'Opening Capacity (MWh)'] * 2)
    max_charge.append(max_c)
    min_charge.append(min_c)
    
    #if float(new_full.loc[i, 'Raw Power (MW)']) < float(max_c[i]):
    #    flags.append(1)
    #elif new_full.loc[i, 'Raw Power (MW)'] > min_c[i]:
    #    flags.append(2)
    #else:
    #    flags.append(0)

fullmodel['Max Charge'] = pd.Series(max_charge)
fullmodel['Max Discharge'] = pd.Series(min_charge)

for i in range(len(fullmodel)):
    if fullmodel.loc[i, 'Raw Power (MW)'] < fullmodel.loc[i, 'Max Charge']:
        flags.append(1)
    elif fullmodel.loc[i, 'Raw Power (MW)'] > fullmodel.loc[i, 'Max Discharge']:
        flags.append(2)
    else:
        flags.append(0)
        
fullmodel['Flags'] = pd.Series(flags)
errors = fullmodel[fullmodel['Flags'] != 0]
errors

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Status,Max Charge,Max Discharge,Flags


In [46]:
fullmodel[fullmodel["Opening Capacity (MWh)"] != fullmodel["Closing Capacity (MWh)"].shift()]

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Status,Max Charge,Max Discharge,Flags
0,2021-07-01 00:30:00,90.51,0.0,0.0,0.0,0.0,Nothing,-300.0,0.0,0


In [47]:
fullmodel.to_excel('../../preprocessed_data/First Algorithm/fullMaximisation(independent).xlsx', index=False)

### Dependency Model

In [53]:
def renameColumns(dataframe, region = 'VIC'):
    if region == 'VIC':
        price_name = 'Regions VIC Trading Price ($/MWh)'
    elif region == 'NSW':
        price_name = 'Regions NSW Trading Price ($/MWh)'
    elif region == 'SA':
        price_name = 'Regions SA Trading Price ($/MWh)'
    elif region == 'TAS':
        price_name = 'Regions TAS Trading Price ($/MWh)'
        
    dataframe.columns = ['Time (UTC+10)', price_name, 'Raw Power (MW)', 'Market Dispatch (MWh)', 'Opening Capacity (MWh)', 'Closing Capacity (MWh)']
    return dataframe

def computeRevenue(dataframe, region = "VIC"):
    region = 'VIC'
    if region == 'VIC':
        price_name = 'Regions VIC Trading Price ($/MWh)'
    elif region == 'NSW':
        price_name = 'Regions NSW Trading Price ($/MWh)'
    elif region == 'SA':
        price_name = 'Regions SA Trading Price ($/MWh)'
    elif region == 'TAS':
        price_name = 'Regions TAS Trading Price ($/MWh)'
        
    dataframe['mlf'] = 0
    dataframe.loc[dataframe['Market Dispatch (MWh)'] > 0, 'mlf'] = 0.991
    dataframe.loc[dataframe['Market Dispatch (MWh)'] < 0, 'mlf'] = 1/0.991
    dataframe['Revenue ($)'] = dataframe[price_name] * dataframe['mlf'] * dataframe['Market Dispatch (MWh)']
    return dataframe['Revenue ($)'].sum()

In [94]:
PERIOD = 48
def DependencyOptimisation(data, timePeriod, region = "VIC"):
    raw_power = []
    market_dispatch = []
    opening_capacity = []
    closing_capacity = []
    
    count = 0
    # Iterate over each timePeriod.
    for time in timePeriod:
        start_t = time[0]
        end_t = time[1]
        period_t = time[2]
        
        # Select the interval
        data_interval = data.loc[(data['Time (UTC+10)'] >= start_t) & \
                                 (data['Time (UTC+10)'] <= end_t)].copy()
        data_interval = data_interval.reset_index(drop = True)
        original_revenue = computeRevenue(data_interval) # Revenue of the previos model/given dataframe.
        
        daily_rev = 0
        # If selected interval period is greater than 48, then that means there is dependency.
        # Compute the revenue of independent model and dependent model, compare revenue with original revenue as well.
        if period_t > PERIOD:
            start = 0
            end = PERIOD 
            tmp_rp = []
            tmp_md = []
            tmp_oc = []
            tmp_cc = []
            while end <= len(data_interval):
                tmp_data = data_interval.iloc[start:end, :].copy()
                tmp_data = tmp_data.reset_index(drop = True)
                # Compute independent maximisation
                daily_data = fullMaximisation(tmp_data, period = len(tmp_data))
                daily_rev += computeRevenue(daily_data)
                tmp_rp.extend(daily_data['Raw Power (MW)'])
                tmp_md.extend(daily_data['Market Dispatch (MWh)'])
                tmp_oc.extend(daily_data['Opening Capacity (MWh)'])
                tmp_cc.extend(daily_data['Closing Capacity (MWh)'])
                
                start += PERIOD
                end += PERIOD
        # Compute dependent maximisation.
        dependent_data = fullMaximisation(data_interval, period = len(data_interval))
        revenue_2 = computeRevenue(dependent_data)
        # If dependent model revenue is higher or independent model revenue is higher than the original
        if (revenue_2 > original_revenue or daily_rev > original_revenue):
            # If there period is more than 48, compare the independent and dependent revenue
            # if independent is higher, settle with independent.
            # else settle with dependent.
            if daily_rev != 0 and daily_rev > revenue_2:
                raw_power.extend(tmp_rp)
                market_dispatch.extend(tmp_md)
                opening_capacity.extend(tmp_oc)
                closing_capacity.extend(tmp_cc)
            else:
                raw_power.extend(dependent_data['Raw Power (MW)'])
                market_dispatch.extend(dependent_data['Market Dispatch (MWh)'])
                opening_capacity.extend(dependent_data['Opening Capacity (MWh)'])
                closing_capacity.extend(dependent_data['Closing Capacity (MWh)'])
        else:
            # Original data
            raw_power.extend(data_interval['Raw Power (MW)'])
            market_dispatch.extend(data_interval['Market Dispatch (MWh)'])
            opening_capacity.extend(data_interval['Opening Capacity (MWh)'])
            closing_capacity.extend(data_interval['Closing Capacity (MWh)'])
            
        print(count) # To show progress of the algorithm
        count += 1
    
    data['Raw Power (MW)'] = pd.Series(raw_power)
    data['Market Dispatch (MWh)'] = pd.Series(market_dispatch)
    data['Opening Capacity (MWh)'] = pd.Series(opening_capacity)
    data['Closing Capacity (MWh)'] = pd.Series(closing_capacity)
    
    return data

def computeRawPower(dataframe):
    tmp_df = dataframe.copy()
    tmp_df['Efficiency'] = 0
    tmp_df.loc[tmp_df['Actual'] > 0, 'Efficiency'] = 2 / 0.9
    tmp_df.loc[tmp_df['Actual'] < 0, 'Efficiency'] = 2
    tmp_df['Raw Power'] = tmp_df['Efficiency'] * tmp_df['Actual']
    dataframe['Raw Power'] = tmp_df['Raw Power']
    return dataframe

In [96]:
# HERE I HAVE REMOVE THE FIRST ENTRY IN THE FILE BEFORE FEEDING IT IN AND
# COMPUTED OUT THE RAW POWER AND REVENUE IN EXCEL
SM = pd.read_excel("../../preprocessed_data/First Algorithm/SM.xlsx")
SM = computeRawPower(SM)
SM = SM[['Time', 'Price', 'Raw Power', 'Actual', 'Opening Capacity', 'Closing Capacity']]

In [97]:
SM

,Time,Price,Raw Power,Actual,Opening Capacity,Closing Capacity
0,2018-01-01 00:30:00,92.46,0.0,0.0,0,0
1,2018-01-01 01:00:00,87.62,0.0,0.0,0,0
2,2018-01-01 01:30:00,73.08,0.0,0.0,0,0
3,2018-01-01 02:00:00,70.18,0.0,0.0,0,0
4,2018-01-01 02:30:00,67.43,0.0,0.0,0,0
...,...,...,...,...,...,...
63451,2021-08-14 22:00:00,49.93,0.0,0.0,580,580
63452,2021-08-14 22:30:00,62.86,0.0,0.0,580,580
63453,2021-08-14 23:00:00,32.26,0.0,0.0,580,580
63454,2021-08-14 23:30:00,25.10,0.0,0.0,580,580


In [78]:
from PeriodMaximisation import renameColumns, getTimePeriod, DependencyOptimisation

SM = pd.read_excel("../../preprocessed_data/First Algorithm/SM.xlsx")
SM = computeRawPower(SM)
SM = SM[['Time', 'Price', 'Raw Power', 'Actual', 'Opening Capacity', 'Closing Capacity']]
SM = renameColumns(SM)
timePeriod = getTimePeriod(SM)
#output = DependencyOptimisation(SM, timePeriod)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [79]:
output.to_excel('../../preprocessed_data/First Algorithm/dependencyOutput.xlsx', index=False)

In [73]:
time = timePeriod[3]
start_t = time[0]
end_t = time[1]
period_t = time[2]

# Select the interval
data_interval = SM.loc[(SM['Time (UTC+10)'] >= start_t) & \
                         (SM['Time (UTC+10)'] <= end_t)].copy()
data_interval = data_interval.reset_index(drop = True)
original_revenue = computeRevenue(data_interval)
dependent_data = fullMaximisation(data_interval, period = len(data_interval))

In [84]:
max_charge = []
min_charge = []
flags = []
for i in range(len(output)):
    max_c = -min(300, (580 - output.loc[i, 'Opening Capacity (MWh)'])/0.9 * 2)
    min_c = min(300, output.loc[i, 'Opening Capacity (MWh)'] * 2)
    max_charge.append(max_c)
    min_charge.append(min_c)
    
    #if float(new_full.loc[i, 'Raw Power (MW)']) < float(max_c[i]):
    #    flags.append(1)
    #elif new_full.loc[i, 'Raw Power (MW)'] > min_c[i]:
    #    flags.append(2)
    #else:
    #    flags.append(0)

output['Max Charge'] = pd.Series(max_charge)
output['Max Discharge'] = pd.Series(min_charge)

for i in range(len(output)):
    if output.loc[i, 'Raw Power (MW)'] < output.loc[i, 'Max Charge']:
        flags.append(1)
    elif output.loc[i, 'Raw Power (MW)'] > output.loc[i, 'Max Discharge']:
        flags.append(2)
    else:
        flags.append(0)
        
output['Flags'] = pd.Series(flags)
errors = output[output['Flags'] != 0]
errors

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Max Charge,Max Discharge,Flags
1805,2018-02-07 15:00:00,428.57,-33.333333,-16.666667,565.0,580.0,-33.333333,300.0,1
8562,2018-06-28 09:30:00,91.51,-33.333333,-16.666667,565.0,580.0,-33.333333,300.0,1
8571,2018-06-28 14:00:00,86.92,-66.666667,-33.333333,550.0,580.0,-66.666667,300.0,1
9374,2018-07-15 07:30:00,32.95,-33.333333,-16.666667,565.0,580.0,-33.333333,300.0,1
13125,2018-10-01 11:00:00,71.73,-33.333333,-16.666667,565.0,580.0,-33.333333,300.0,1
...,...,...,...,...,...,...,...,...,...
60471,2021-06-13 20:00:00,63.13,-66.666667,-33.333333,550.0,580.0,-66.666667,300.0,1
60650,2021-06-17 13:30:00,140.36,-33.333333,-16.666667,565.0,580.0,-33.333333,300.0,1
61838,2021-07-12 07:30:00,55.09,-33.333333,-16.666667,565.0,580.0,-33.333333,300.0,1
62625,2021-07-28 17:00:00,23.49,-33.333333,-16.666667,565.0,580.0,-33.333333,300.0,1


In [85]:
errors.to_excel('../../preprocessed_data/First Algorithm/ERRORSlocalMaximisation.xlsx', index=False)

In [81]:
output.to_excel('../../preprocessed_data/First Algorithm/dependencyOutput.xlsx', index=False)

In [82]:
output[output["Opening Capacity (MWh)"] != output["Closing Capacity (MWh)"].shift()]


,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh),Max Charge,Max Discharge,Flags
0,2018-01-01 00:30:00,92.46,0.0,0.0,0.0,0.0,-300.0,0.0,0


### 128

In [87]:
data_128 = pd.read_excel("../../preprocessed_data/First Algorithm/128.xlsx")
data_128 = data_128[['Time', 'Price', 'Raw Power', 'Actual', 'Opening Capacity', 'Closing Capacity']]
data_128 = renameColumns(data_128)
timePeriod = getTimePeriod(data_128)
data_128

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh)
0,2018-01-01 00:30:00,92.46,0.0,0.0,0.0,0.0
1,2018-01-01 01:00:00,87.62,0.0,0.0,0.0,0.0
2,2018-01-01 01:30:00,73.08,0.0,0.0,0.0,0.0
3,2018-01-01 02:00:00,70.18,0.0,0.0,0.0,0.0
4,2018-01-01 02:30:00,67.43,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
63451,2021-08-14 22:00:00,49.93,-300.0,-150.0,0.0,135.0
63452,2021-08-14 22:30:00,62.86,270.0,121.5,135.0,0.0
63453,2021-08-14 23:00:00,32.26,0.0,0.0,0.0,0.0
63454,2021-08-14 23:30:00,25.10,0.0,0.0,0.0,0.0


In [88]:
output_128 = DependencyOptimisation(data_128, timePeriod)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [89]:
output_128.to_excel('../../preprocessed_data/First Algorithm/dependencyOutput_128.xlsx', index=False)

In [98]:
test = pd.read_excel('../../preprocessed_data/First Algorithm/fullMaximisation(independent).xlsx')
test_timePeriod = getTimePeriod(test)

IndexError: list index out of range

In [158]:
PERIOD = 48
# Get dependency on the given dataframe.
# If there is capacity in the end of the day and the beginning of the day that counts as dependency
def getDependency(dataframe):
    dayBorder = []
    
    dependency = []
    for i in range(PERIOD, len(dataframe), PERIOD):
        dayBorder.append((i - 1, i))
        # If the closing capacity of last period of the day are not zero and the opening capacity the following day are 
        # not empty, then there is dependency between the days. Therefore, append.
        if dataframe.loc[i - 1, 'Closing Capacity (MWh)'] != 0 and dataframe.loc[i, 'Opening Capacity (MWh)'] != 0:
            dependency.append((i - 1, i))
    
    consecutiveDependency = []
    tmp = []
    for d in range(len(dependency) - 1):
        curr_d = dependency[d][0]
        next_d = dependency[d + 1][0]
        tmp.append(dependency[d])
        if (next_d - curr_d != PERIOD):
            consecutiveDependency.append(tmp)
            tmp = []
    
    if len(dependency) != 0 and dependency[-1][0] - consecutiveDependency[-1][0][0] == PERIOD:
        consecutiveDependency[-1].append(dependency[-1])
    else:
        consecutiveDependency.append([dependency[-1]])
        
    # Selects only the the beginning and last period of the consecutive dependencies.
    # For example, (monday, tuesday, wednesday, thursday, friday). So select (monday, friday) 
    for c in range(len(consecutiveDependency)):
        current = consecutiveDependency[c]
        if len(current) != 1:
            consecutiveDependency[c] = [current[0], current[-1]]
            
    return consecutiveDependency

# A function to create the time period given the index
def createPeriod(current, until):
    tmp = []
    for i in range(current, until, PERIOD):
        tmp.append((i, i + PERIOD - 1))
    return tmp


# Get the timeperiod of all the data including the dependencies.
def getTimePeriod(dataframe):
    dependency = getDependency(dataframe)
    tmp = []
    for depend in dependency:
        openPeriod = depend[0][0] - PERIOD + 1     # 48th period - 47 = 1st period
        closePeriod = depend[-1][-1] + PERIOD - 1  # 1st period + 47 = 48th period
        tmp.append((openPeriod, closePeriod))
    
    last_dependency = dependency[-1][0][-1]

    timeIndex = []
    timeIndex.extend(createPeriod(0, tmp[0][0]))
    
    for t in range(1, len(tmp)):
        timeIndex.append((tmp[t-1]))
        timeIndex.extend(createPeriod(tmp[t-1][1] + 1, tmp[t][0]))
    timeIndex.append(tmp[t])  
    
    last_index = timeIndex[-1][-1]
    len_df = len(dataframe)
    current = last_index
    
    while current != len_df - 1:
        timeIndex.append((current + 1, current + 1 + 47))
        current = timeIndex[-1][-1]

    timePeriod = []
    for index in timeIndex:
        timePeriod.append((dataframe.iloc[index[0], 0], dataframe.iloc[index[1], 0], index[1] - index[0] + 1))
  
    return timePeriod

In [180]:
pred_df = pd.read_excel('../../preprocessed_data/First Algorithm/Pred.xlsx')
pred_df.drop(index=pred_df.index[0], axis=0, inplace=True)
pred_df = pred_df.reset_index(drop = True)
pred_df

,Unnamed: 0,Time,Price,Status,Actual,Restrict,Opening Capacity,Closing Capacity,Period,ROCNACharge,ROCNADischarge,Shift,mlf,Revenue,Power
0,1,2018-01-01 00:30:00,92.46,Do Nothing,0.0,0,0,0,1,1.0,NaN,True,0.991,0.0,0.0
1,2,2018-01-01 01:00:00,87.62,Do Nothing,0.0,0,0,0,1,1.0,NaN,True,0.991,0.0,0.0
2,3,2018-01-01 01:30:00,73.08,Do Nothing,0.0,0,0,0,1,1.0,NaN,True,0.991,0.0,0.0
3,4,2018-01-01 02:00:00,70.18,Do Nothing,0.0,0,0,0,1,1.0,NaN,True,0.991,0.0,0.0
4,5,2018-01-01 02:30:00,67.43,Do Nothing,0.0,0,0,0,1,1.0,NaN,True,0.991,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63451,63452,2021-08-14 22:00:00,49.93,Do Nothing,0.0,0,580,580,24022,5090.0,NaN,False,0.991,0.0,0.0
63452,63453,2021-08-14 22:30:00,62.86,Do Nothing,0.0,0,580,580,24022,5090.0,NaN,False,0.991,0.0,0.0
63453,63454,2021-08-14 23:00:00,32.26,Do Nothing,0.0,0,580,580,24022,5090.0,NaN,False,0.991,0.0,0.0
63454,63455,2021-08-14 23:30:00,25.10,Do Nothing,0.0,0,580,580,24022,5090.0,NaN,False,0.991,0.0,0.0


In [181]:
pred_df = computeRawPower(pred_df)
tmp_pred_df = pred_df[['Time', 'Price', 'Raw Power', 'Actual', 'Opening Capacity', 'Closing Capacity']].copy()
tmp_pred_df = renameColumns(tmp_pred_df)
pred_timePeriod = getTimePeriod(tmp_pred_df)
pred_timePeriod

[(Timestamp('2018-01-01 00:30:00'), Timestamp('2018-01-02 00:00:00'), 48),
 (Timestamp('2018-01-02 00:30:00'), Timestamp('2018-01-06 00:00:00'), 192),
 (Timestamp('2018-01-06 00:30:00'), Timestamp('2018-01-09 00:00:00'), 144),
 (Timestamp('2018-01-09 00:30:00'), Timestamp('2018-01-10 00:00:00'), 48),
 (Timestamp('2018-01-10 00:30:00'), Timestamp('2018-01-12 00:00:00'), 96),
 (Timestamp('2018-01-12 00:30:00'), Timestamp('2018-01-13 00:00:00'), 48),
 (Timestamp('2018-01-13 00:30:00'), Timestamp('2018-01-14 00:00:00'), 48),
 (Timestamp('2018-01-14 00:30:00'), Timestamp('2018-01-18 00:00:00'), 192),
 (Timestamp('2018-01-18 00:30:00'), Timestamp('2018-01-19 00:00:00'), 48),
 (Timestamp('2018-01-19 00:30:00'), Timestamp('2018-01-21 00:00:00'), 96),
 (Timestamp('2018-01-21 00:30:00'), Timestamp('2018-01-27 00:00:00'), 288),
 (Timestamp('2018-01-27 00:30:00'), Timestamp('2018-01-30 00:00:00'), 144),
 (Timestamp('2018-01-30 00:30:00'), Timestamp('2018-01-31 00:00:00'), 48),
 (Timestamp('2018-01

In [182]:
output_pred = DependencyOptimisation(tmp_pred_df, pred_timePeriod)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [183]:
output_pred

,Time (UTC+10),Regions VIC Trading Price ($/MWh),Raw Power (MW),Market Dispatch (MWh),Opening Capacity (MWh),Closing Capacity (MWh)
0,2018-01-01 00:30:00,92.46,0.0,0.0,0.0,0.0
1,2018-01-01 01:00:00,87.62,0.0,0.0,0.0,0.0
2,2018-01-01 01:30:00,73.08,0.0,0.0,0.0,0.0
3,2018-01-01 02:00:00,70.18,0.0,0.0,0.0,0.0
4,2018-01-01 02:30:00,67.43,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
63451,2021-08-14 22:00:00,49.93,-300.0,-150.0,0.0,135.0
63452,2021-08-14 22:30:00,62.86,270.0,121.5,135.0,0.0
63453,2021-08-14 23:00:00,32.26,0.0,0.0,0.0,0.0
63454,2021-08-14 23:30:00,25.10,0.0,0.0,0.0,0.0


In [184]:
computeRevenue(output_pred)

128803035.894666

In [186]:
output_pred.to_excel('../../preprocessed_data/First Algorithm/pred_dependency.xlsx', index=False)